Training Data

In [1]:
# Lab 5 Logistic Regression Classifier
import tensorflow as tf
import numpy as np
tf.set_random_seed(743)  # for reproducibility

# https://www.kaggle.com/primaryobjects/voicegender
file_queue = tf.train.string_input_producer(
    ['voice.csv'], name='file_queue')

reader = tf.TextLineReader()
_, value = reader.read(file_queue)

print(value)

Tensor("ReaderReadV2:1", shape=(), dtype=string)


In [ ]:

conv_dic = { b'male': 1., b'female' : 0.}
record_defaults = [[0.], [0.], [0.], [0.], [0.],
                   [0.], [0.], [0.], [0.], [0.],
                   [0.], [0.], [0.], [0.], [0.],
                   [0.], [0.], [0.], [0.], [0.], [b'male']]
xy = tf.decode_csv(value, record_defaults=record_defaults)

sess = tf.Session()
print(sess.run(value))

# collect batches of csv in
train_x_batch, train_y_batch = \
    tf.train.batch([xy[2:6], xy[-1:]], batch_size=32)

build a model

In [12]:
X = tf.placeholder(tf.float32, shape=[None, 4])
Y = tf.placeholder(tf.float32, shape=[None, 1])
W = tf.Variable(tf.random_normal([4, 1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

# Hypothesis using sigmoid: tf.div(1., 1. + tf.exp(tf.matmul(X, W)))
hypothesis = tf.sigmoid(tf.matmul(X, W) + b)

cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) * tf.log(1 - hypothesis))

train = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost)

predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))

train a model

In [13]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    x_data, y_data = sess.run([train_x_batch, train_y_batch])
    for step in range(4001):
        x_data, y_data = sess.run([train_x_batch, train_y_batch])
        for i in range(32):
            y_data[i][0] = conv_dic[y_data[i][0]]
        cost_val, _ = sess.run([cost, train], feed_dict={X: x_data, Y: y_data})
        if step % 200 == 0:
            print(step, cost_val)
    
    for step in range(11):
        # accuracy report
        x_data, y_data = sess.run([train_x_batch, train_y_batch])
        for i in range(32):
            y_data[i][0] = conv_dic[y_data[i][0]]
        h, c, a = sess.run([hypothesis, predicted, accuracy], feed_dict={X: x_data, Y: y_data})
        if step % 1 == 0:
            print("Accuracy: ", a)

    coord.request_stop()
    coord.join(threads)

0 1.5677602
200 1.1556385
400 0.94385254
600 0.80007625
800 0.7817553
1000 0.7382495
1200 0.7358403
1400 0.7022146
1600 0.6574004
1800 0.65852153
2000 0.6671367
2200 0.66944045
2400 0.6789074
2600 0.67196184
2800 0.6651708
3000 0.6688529
3200 0.6601175
3400 0.65097696
3600 0.6434678
3800 0.6305413
4000 0.63463455
Accuracy:  1.0
Accuracy:  1.0
Accuracy:  1.0
Accuracy:  1.0
Accuracy:  0.9375
Accuracy:  1.0
Accuracy:  1.0
Accuracy:  0.78125
Accuracy:  0.40625
Accuracy:  0.3125
Accuracy:  0.0625
